### Chatbot Prompt Engineering

Notebook for testing engineered prompts on LikeHome chatbot.

#### Enter Persona

Chatbot will use this for prompt to guide it through any prompt. See examples below. 

Add a new string to the `PERSONAS` list and run the remaining cells. Variables for `current_date` and `user_info` are also available to help user craft responses.

In [ ]:
from datetime import date

current_date = date.today()

# fake user details to test account retrieval ability
user_info = '''{
"name": "Bob Jones",
"reward_points": 50,
"email": bobjones@hotmail.com,
}'''

PERSONAS: list = [
# prompt 0, works consistency with most situations
f'''Your name is Nexus and you are a chatbot for a hotel reservation website called LikeHome.
The date today is {current_date} for reference.

Respond to users if they ask about hotels or their account. Parse their queries to extract field details. 
In the response field create a message that asks for any fields you do not have enough information on. 
Default the following fields and change them if users have preference: radius=20, min_rate=0, max_rate=1000.
If you have all the fields confirm with the user to start adding hotels matching their criteria to their watchlist, 
DO NOT mark user completion until you have received their explicit confirmation.
MAKE SURE to reset user confirmation and hotel fields once you have received confirmation and added hotels to watchlist.
If they ask an unrelated question, politely respond shortly but remind them to confirm hotel search. 

Here is all the information on the users account if they ask about it: {user_info}''',

# TODO add additional prompts to test
]

#### Set-Up

Run the below cells to set up chat gpt requests.

In [ ]:
from openai import OpenAI
import numpy as np
import pandas as pd
from IPython.display import display
from pydantic import BaseModel
import json

hotel_search_results = [
    {
        "code": 13528,
        "name": "Hotel Spero",
        "minRate": "107.56",
        "maxRate": "549.41",
        "currency": "EUR",
        "description": "Hotel Spero at Union Square is in Union Square, a walkable area in San Francisco with good shopping. Cable Car Museum and Asian Art Museum of San Francisco are cultural highlights, and some of the area's notable landmarks include Dragon's Gate and Lombard Street. Hotel Spero at Union Square provides dry cleaning/laundry services, a bar, and a 24-hour gym. The on-site American cuisine restaurant, Jaspers Corner Tap, offers breakfast, brunch, lunch, dinner, and happy hour. Stay connected with in-room WiFi, with speed of 100+ Mbps (good for 1\u20132 people or up to 6 devices), and guests can find other amenities such as a 24-hour business center. Resort fee to be collected at time of check-in. \n\nSan Francisco Museum of Modern Art - 13 min walk\nMoscone Convention Center - 16 min walk\nLombard Street - 3 min drive\nFerry Building - 3 min drive\nOracle Park - 4 min drive\n\nRestaurants\nCityscape Bar and Restaurant - 3 min walk\nPinecrest Diner - 3 min walk\nBourbon & Branch - 2 min walk\nThe Halal Guys - 1 min walk\nJones - 3 min walk\n\nOther perks include:\nFull breakfast (surcharge), smoke-free premises, and a 24-hour front desk\nA banquet hall, 5 meeting rooms, and luggage storage\nA porter/bellhop, tour/ticket assistance, and ATM/banking services\n\nRoom features\nAll 236 rooms offer comforts such as air conditioning, in addition to thoughtful touches like safes and WiFi. Guest reviews speak well of the clean, comfortable rooms at the property. \nOther conveniences in all rooms include:\nBathrooms with eco-friendly toiletries and shower/tub combinations\nWardrobes/closets, recycling, and coffee/tea makers\n\nOptional extras\nFee for full breakfast: approximately USD 22 for adults and USD 22 for children\nNearby parking fee: USD 49 per night (20 ft away)\nPet fee: USD 75 per pet, per stay\nService animals are exempt from fees\nEarly check-in is available for a fee (subject to availability)\nLate check-out is available for a fee (subject to availability)",
        "city": "SAN FRANCISCO",
        "address": "405 Taylor Street",
        "email": "N/A",
        "web": "N/A",
        "phone": "+14158852500",
        "images": [
            "http://photos.hotelbeds.com/giata/xxl/01/013528/013528a_hb_ba_001.jpg",
            "http://photos.hotelbeds.com/giata/xxl/01/013528/013528a_hb_ro_056.jpg",
            "http://photos.hotelbeds.com/giata/xxl/01/013528/013528a_hb_a_015.jpg",
            "http://photos.hotelbeds.com/giata/xxl/01/013528/013528a_hb_r_016.jpg"
        ],
        "rooms": [
            {
                "name": "Two Double Room",
                "netRate": "107.56",
                "adults": 2,
                "children": 2
            },
            {
                "name": "City View Two Double Room",
                "netRate": "116.12",
                "adults": 2,
                "children": 2
            },
            {
                "name": "Queen Room",
                "netRate": "119.34",
                "adults": 2,
                "children": 2
            },
            {
                "name": "DOUBLE DELUXE KING BED",
                "netRate": "119.34",
                "adults": 2,
                "children": 2
            }
        ]
    },
    {
        "code": 51575,
        "name": "Holiday Inn Express Hotel & Suites Oakland",
        "minRate": "109.01",
        "maxRate": "109.01",
        "currency": "EUR",
        "description": "Holiday Inn Express Hotel & Suites Oakland-Airport, an IHG Hotel is in an area with good airport proximity. Fox Theater and Greek Theater are cultural highlights, and some of the area's activities can be experienced at Berkeley Marina and Ferry Building. Consider a stay at Holiday Inn Express Hotel & Suites Oakland-Airport, an IHG Hotel and take advantage of a free breakfast buffet, dry cleaning/laundry services, and a 24-hour gym. In addition to a 24-hour business center, guests can connect to free in-room WiFi. All 95 rooms include comforts such as air conditioning, in addition to thoughtful touches like free WiFi and safes. An ordinance fee of $11. 40 will be collected by hotel at time of check-in. \n\nOracle Arena - 4 min drive\nRolling Loud Festival - 4 min drive\nRingCentral Coliseum - 4 min drive\nOakland Zoo - 8 min drive\nSan Francisco Bay - 14 min drive\n\nRestaurants\nRaising Cane's Chicken Fingers - 18 min walk\nIn-N-Out Burger - 2 min drive\nJack In The Box - 2 min drive\nStarbucks - 18 min walk\nStarbucks - 9 min walk\n\nAdditional perks include:\nSelf parking (surcharge), a roundtrip airport shuttle (surcharge), and an electric car charging station\n\nPets not allowed",
        "city": "OAKLAND",
        "address": "66 Airport Access Road",
        "email": "N/A",
        "web": "N/A",
        "phone": "+16503556300",
        "images": [
            "http://photos.hotelbeds.com/giata/xxl/05/051575/051575a_hb_ro_011.jpg",
            "http://photos.hotelbeds.com/giata/xxl/05/051575/051575a_hb_ro_004.jpg",
            "http://photos.hotelbeds.com/giata/xxl/05/051575/051575a_hb_a_010.jpg",
            "http://photos.hotelbeds.com/giata/xxl/05/051575/051575a_hb_r_007.jpg"
        ],
        "rooms": [
            {
                "name": "2 Queen Beds Standard",
                "netRate": "109.01",
                "adults": 2,
                "children": 2
            }
        ]
    },
    {
        "code": 107904,
        "name": "Knotts Berry Farm Hotel",
        "minRate": "101.54",
        "maxRate": "406.18",
        "currency": "EUR",
        "description": "The comfortable Knotts Berry Farm Hotel is conveniently situated in Buena Park, within the Knotts Berry Farm theme park. It features a  a restaurant with a daily visits from Snoopy. Knott's Theme Park and Soak City Water Park are just steps away; the hotel offers a complimentary shuttle service to the Disneyland Resort. Other attractions such as the Anaheim Convention Center, the Honda Center, home of the NHL team Mighty Ducks of Anaheim or the Angel stadium, home of the MLB team California Angels, are within a short drive.\n\nReferbishment-\nUpdates as of 5/10/2023:\n-Starting Thursday, May 11th, the pool will be closed Monday through Friday. \nThe pool will reopen every Friday at 6pm and will remain open through Sunday.\n-A limited guest room service menu is still available. Room service can be ordered between 4:00 PM and 10:00 PM daily for a limited time. Please contact (714) 243-2010 for more information.\n-The hotel entrance is currently unavailable. Please follow the directional signs displayed in the parking lot.\n-The Front Desk has relocated to the north side of the hotel.\n-Guests can enter utilizing the pool entrance or the north entrance.\n-The gift shop is currently unavailable. Guests can visit our Marketplace for alternative shopping locations.\n-We no longer have a sports court available for guest use.\n-The hotel restaurant is currently unavailable. Guests can visit our Marketplace for Chicken to Go, Chicken Dinner Restaurant, or TGI Fridays.\n-The breakfast buffet has been temporarily relocated to the Chicken Dinner Restaurant. The buffet will only be available daily from 6:30 AM to 10:30 AM for hotel guests. Breakfast vouchers are available for purchase through select packages or purchased onsite. (Must present hotel room keys when purchasing onsite) \n-The lobby area is currently unavailable.",
        "city": "BUENA PARK",
        "address": "7675 Crescent Avenue",
        "email": "N/A",
        "web": "N/A",
        "phone": "+17149951111",
        "images": [
            "http://photos.hotelbeds.com/giata/xxl/10/107904/107904a_hb_r_001.jpg",
            "http://photos.hotelbeds.com/giata/xxl/10/107904/107904a_hb_f_003.jpg",
            "http://photos.hotelbeds.com/giata/xxl/10/107904/107904a_hb_ro_014.jpg",
            "http://photos.hotelbeds.com/giata/xxl/10/107904/107904a_hb_ro_013.jpg"
        ],
        "rooms": [
            {
                "name": "DOUBLE TWO QUEEN BEDS",
                "netRate": "101.54",
                "adults": 2,
                "children": 1
            }
        ]
    },
]
empty_search_results = []
search_results_prompt = f'''hotel results in json list format: {empty_search_results}.
Tell the user how many hotels we were able to find and a brief description of each one. If it is an empty list, tell the user you were 
unable to add any hotels to their watchlist and they can try booking again with you or modify it themselves in search tab.'''
convos = []
for persona in PERSONAS:
    convos.append([{"role": "system", "content": persona}])

client = OpenAI()

# define JSON format for model base outputs
class SearchFilter(BaseModel):
    location: str
    check_in: str
    check_out: str
    adults: int
    children: int
    radius: int
    min_price: int
    max_price: int
    response: str
    user_search_confirmation: bool

class SearchResponse(BaseModel):
    response: str

def reg_chat(messages: list):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=messages,
        response_format=SearchResponse,
        temperature=0.2,
    )
    return json.loads(completion.choices[0].message.content)

def hotel_chat(messages: list):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=messages,
        response_format=SearchFilter,
        temperature=0.2,
    )
    response = completion.choices[0].message.content
    json_response = json.loads(response)
    messages.append({"role": "assistant", "content": response})
    if (json_response['user_search_confirmation']):
        messages.append({"role": "system", 
                         "content": search_results_prompt})
        search_response = reg_chat(messages)['response']
        print("SEARCH RESPONSE: " +  search_response)
        json_response['search_response'] = search_response   # TODO with how it is done in real app in writing response to db

    return json.dumps(json_response, indent=4)

def main():
    global convos
    while (True):
        count = 0
        user_prompt = input("Your a prompt: ")
        if user_prompt == "quit":
            print("chat ended")
            break
        for convo in convos:
            convo.append({"role": "user", "content": user_prompt})
            print("-------------------------------")
            print("Using PERSONA at index %d" % count) 
            print(hotel_chat(convo))
            count += 1
        print(f'''-------------------------------\n''')

#### Run the chatbot

Run the below cell to test the chatbot. It will prompt u to enter a prompt. Enter **quit** to end the chat.

Chatbot will respond in following format:
```
{
    "location": "",
    "check_in": "",
    "check_out": "",
    "adults": 0,
    "children": 0,
    "radius": 0,
    "min_price": 0,
    "max_price": 0,
    "response": "response",
    "user_search_confirmation": false
}
```

look at the `response` field and evaluate how well it is doing, this is verbatim what the user will see when prompting the chatbot. Also ensure `user_confirmation` is only set to true when all fields look accurate. Once `user_confirmation` is true it will trigger creating a watchlist item in the actual app, so we want to make sure we don't do this prematurely.

In [ ]:
main()